# NYC Motor Vehicle Collisions - Data Engineering Project

## Milestone 1: Cleaning, Integration, and Visualization

**This notebook is ready for Google Colab!** Simply upload this file to Colab and run all cells.

This notebook contains:
- Exploratory Data Analysis (EDA)
- Pre-integration data cleaning
- Data integration with related datasets
- Post-integration cleaning
- Visualizations and insights

---

## Table of Contents
1. [Data Loading](#data-loading)
2. [Exploratory Data Analysis](#eda)
3. [Pre-Integration Cleaning](#pre-cleaning)
4. [Data Integration](#integration)
5. [Post-Integration Cleaning](#post-cleaning)
6. [Visualizations](#visualizations)
7. [Research Questions Analysis](#research-questions)



## 1. Data Loading

Load the datasets from NYC Open Data API.


In [1]:
# Install required packages (run this first in Colab if needed)
# !pip install pandas numpy matplotlib seaborn plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style (try different styles if seaborn-v0_8 doesn't work)
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


Libraries imported successfully!
Pandas version: 2.2.2
NumPy version: 2.0.2


In [2]:
# Utility functions for data loading and processing
from pathlib import Path
import re

# Data URLs
CRASHES_URL = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=download'
PERSONS_URL = 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=download'
VEHICLES_URL = 'https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=download'

def load_crashes_data(use_cache=False, sample_size=None):
    """Load crashes dataset."""
    print("Loading crashes data from NYC Open Data...")
    df = pd.read_csv(CRASHES_URL, low_memory=False)
    if sample_size and len(df) > sample_size:
        print(f"Sampling {sample_size} records for faster processing...")
        df = df.sample(n=sample_size, random_state=42)
    print(f"Loaded {len(df):,} crash records.")
    return df

def load_persons_data(use_cache=False):
    """Load persons dataset."""
    print("Loading persons data from NYC Open Data...")
    df = pd.read_csv(PERSONS_URL, low_memory=False)
    print(f"Loaded {len(df):,} person records.")
    return df

def standardize_borough(borough):
    """Standardize borough names."""
    if pd.isna(borough):
        return 'UNKNOWN'
    borough = str(borough).strip().upper()
    mapping = {
        'MANHATTAN': 'MANHATTAN', 'MN': 'MANHATTAN',
        'BROOKLYN': 'BROOKLYN', 'BK': 'BROOKLYN',
        'BRONX': 'BRONX', 'BX': 'BRONX',
        'QUEENS': 'QUEENS', 'QN': 'QUEENS',
        'STATEN ISLAND': 'STATEN ISLAND', 'SI': 'STATEN ISLAND', 'RICHMOND': 'STATEN ISLAND'
    }
    return mapping.get(borough, borough)

def clean_contributing_factor(factor):
    """Clean and standardize contributing factor values."""
    if pd.isna(factor):
        return 'UNSPECIFIED'
    factor = str(factor).strip().upper()
    factor = re.sub(r'\s+', ' ', factor)
    if 'UNSPECIFIED' in factor or 'NOT STATED' in factor or factor == '':
        return 'UNSPECIFIED'
    return factor

def detect_outliers_iqr(series, multiplier=1.5):
    """Detect outliers using IQR method."""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return (series < lower_bound) | (series > upper_bound)

def validate_coordinates(lat, lon):
    """Validate latitude and longitude for NYC area."""
    NYC_LAT_MIN, NYC_LAT_MAX = 40.4, 40.9
    NYC_LON_MIN, NYC_LON_MAX = -74.3, -73.7
    if pd.isna(lat) or pd.isna(lon):
        return False
    return (NYC_LAT_MIN <= lat <= NYC_LAT_MAX and NYC_LON_MIN <= lon <= NYC_LON_MAX)

def calculate_crash_rate_per_capita(df, borough_col='BOROUGH', population_data=None):
    """Calculate crash rate per capita by borough."""
    if population_data is None:
        population_data = {
            'MANHATTAN': 1694251, 'BROOKLYN': 2736074, 'QUEENS': 2405464,
            'BRONX': 1472654, 'STATEN ISLAND': 495747
        }
    crash_counts = df[borough_col].value_counts()
    rates = {}
    for borough, count in crash_counts.items():
        pop = population_data.get(borough.upper(), 1)
        rates[borough] = (count / pop) * 100000
    return pd.Series(rates)

print("Utility functions defined!")


Utility functions defined!


In [3]:
# Load crashes dataset
# Note: For faster processing during development, we can use a sample
# Remove sample_size parameter for full dataset (WARNING: Full dataset is 2M+ records)
df_crashes = load_crashes_data(use_cache=False, sample_size=100000)

print(f"Crashes dataset shape: {df_crashes.shape}")
print(f"\nColumns: {list(df_crashes.columns)}")
df_crashes.head()


Loading crashes data from NYC Open Data...
Sampling 100000 records for faster processing...
Loaded 100,000 crash records.
Crashes dataset shape: (100000, 29)

Columns: ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
2139416,11/08/2024,15:45,BROOKLYN,11201,40.687363,-73.990036,"(40.687363, -73.990036)",SMITH ST,DEAN ST,NaN,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Passing or Lane Usage Improper,NaN,NaN,NaN,4770105,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN
506794,11/10/2019,14:00,QUEENS,11366,40.729270,-73.788290,"(40.72927, -73.78829)",NaN,NaN,75-44 181 STREET,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4238500,Sedan,Sedan,NaN,NaN,NaN
625369,04/05/2019,14:00,NaN,NaN,40.704304,-73.803150,"(40.704304, -73.80315)",90 AVENUE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Turning Improperly,Unspecified,NaN,NaN,NaN,4109437,Box Truck,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
122086,03/09/2022,11:14,MANHATTAN,10021,0.000000,0.000000,"(0.0, 0.0)",NaN,NaN,218 EAST 74 STREET,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4508714,Convertible,Sedan,NaN,NaN,NaN
1215418,09/17/2016,23:15,NaN,NaN,40.704388,-73.994576,"(40.704388, -73.994576)",BROOKLYN BRIDGE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3522798,Sedan,NaN,NaN,NaN,NaN


In [6]:
# Load persons dataset
df_persons = load_persons_data(use_cache=False)

print(f"Persons dataset shape: {df_persons.shape}")
print(f"\nColumns: {list(df_persons.columns)}")
df_persons.head()


Loading persons data from NYC Open Data...
Loaded 5,819,475 person records.
Persons dataset shape: (5819475, 21)

Columns: ['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'PERSON_ID', 'PERSON_TYPE', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION', 'EMOTIONAL_STATUS', 'BODILY_INJURY', 'POSITION_IN_VEHICLE', 'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'COMPLAINT', 'PED_ROLE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2', 'PERSON_SEX']


,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX
0,10249006,4229554,10/26/2019,9:43,31aa2bc0-f545-444f-8cdb-f1cb5cf00b89,Occupant,Unspecified,19141108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,U
1,10255054,4230587,10/25/2019,15:15,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,Unspecified,19144075.0,33.0,Not Ejected,Does Not Apply,Does Not Apply,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F
2,10253177,4230550,10/26/2019,17:55,ae48c136-1383-45db-83f4-2a5eecfb7cff,Occupant,Unspecified,19143133.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,M
3,6650180,3565527,11/21/2016,13:05,2782525,Occupant,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notified Person,NaN,NaN,NaN
4,10255516,4231168,10/25/2019,11:16,e038e18f-40fb-4471-99cf-345eae36e064,Occupant,Unspecified,19144329.0,7.0,Not Ejected,Does Not Apply,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Lap Belt,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F


## 2. Exploratory Data Analysis (EDA)

### 2.1 Dataset Overview


In [7]:
# Basic information about crashes dataset
print("=== CRASHES DATASET OVERVIEW ===")
print(f"Shape: {df_crashes.shape}")
print(f"Memory usage: {df_crashes.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nData types:\n{df_crashes.dtypes.value_counts()}")
print(f"\nMissing values:\n{df_crashes.isnull().sum().sort_values(ascending=False).head(15)}")


=== CRASHES DATASET OVERVIEW ===
Shape: (100000, 29)
Memory usage: 93.88 MB

Data types:
object     18
int64       7
float64     4
Name: count, dtype: int64

Missing values:
VEHICLE TYPE CODE 5              99532
CONTRIBUTING FACTOR VEHICLE 5    99520
VEHICLE TYPE CODE 4              98384
CONTRIBUTING FACTOR VEHICLE 4    98322
VEHICLE TYPE CODE 3              93048
CONTRIBUTING FACTOR VEHICLE 3    92742
OFF STREET NAME                  82078
CROSS STREET NAME                38595
ZIP CODE                         30783
BOROUGH                          30771
ON STREET NAME                   22186
VEHICLE TYPE CODE 2              20183
CONTRIBUTING FACTOR VEHICLE 2    16196
LOCATION                         10766
LATITUDE                         10766
dtype: int64


In [8]:
# Basic statistics for numeric columns
numeric_cols = df_crashes.select_dtypes(include=[np.number]).columns
print("=== NUMERIC COLUMNS STATISTICS ===")
df_crashes[numeric_cols].describe()


=== NUMERIC COLUMNS STATISTICS ===


,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
count,89234.000000,89234.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,40.595397,-73.697505,0.324270,0.001600,0.059440,0.000820,0.029040,0.000150,0.231620,0.000610,3.270651e+06
std,2.283035,4.293773,0.701387,0.041683,0.248531,0.030321,0.170285,0.012247,0.663835,0.025488,1.504194e+06
min,0.000000,-201.237060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.500000e+01
25%,40.667175,-73.974570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.193336e+06
50%,40.720280,-73.927120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.744592e+06
75%,40.769333,-73.866582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.301969e+06
max,42.318317,0.000000,24.000000,3.000000,6.000000,3.000000,3.000000,1.000000,24.000000,2.000000,4.857103e+06


In [9]:
# Check for duplicates
print(f"Total rows: {len(df_crashes)}")
print(f"Duplicate rows: {df_crashes.duplicated().sum()}")
print(f"Duplicate COLLISION_ID: {df_crashes['COLLISION_ID'].duplicated().sum()}")


Total rows: 100000
Duplicate rows: 0
Duplicate COLLISION_ID: 0


### 2.2 Initial Visualizations


In [10]:
# Missing values visualization
missing_data = df_crashes.isnull().sum().sort_values(ascending=False).head(15)
fig = px.bar(
    x=missing_data.values,
    y=missing_data.index,
    orientation='h',
    title='Top 15 Columns with Missing Values',
    labels={'x': 'Number of Missing Values', 'y': 'Column'}
)
fig.update_layout(height=500)
fig.show()


In [11]:
# Distribution of crashes by borough (before cleaning)
if 'BOROUGH' in df_crashes.columns:
    borough_counts = df_crashes['BOROUGH'].value_counts()
    fig = px.bar(
        x=borough_counts.index,
        y=borough_counts.values,
        title='Crashes by Borough (Raw Data)',
        labels={'x': 'Borough', 'y': 'Number of Crashes'}
    )
    fig.update_layout(height=400)
    fig.show()


## 3. Pre-Integration Data Cleaning

### 3.1 Handle Missing Values

**Strategy:**
- **Drop**: Records with missing critical fields (COLLISION_ID, CRASH_DATE)
- **Impute**: Use mode/median for categorical/numeric fields where appropriate
- **Keep as 'UNKNOWN'**: For categorical fields where missing is meaningful


In [16]:
# Standardize crash date column name
df_crashes = df_crashes.rename(columns={"CRASH DATE": "CRASH_DATE"})


# Create a copy for cleaning
df_crashes_clean = df_crashes.copy()

print(f"Original shape: {df_crashes_clean.shape}")

# Drop rows with missing COLLISION_ID (critical field)
df_crashes_clean = df_crashes_clean.dropna(subset=['COLLISION_ID'])
print(f"After dropping missing COLLISION_ID: {df_crashes_clean.shape}")

# Drop rows with missing CRASH_DATE (critical field)
df_crashes_clean = df_crashes_clean.dropna(subset=['CRASH_DATE'])
print(f"After dropping missing CRASH_DATE: {df_crashes_clean.shape}")

# Convert CRASH_DATE to datetime
df_crashes_clean['CRASH_DATE'] = pd.to_datetime(df_crashes_clean['CRASH_DATE'], errors='coerce')
df_crashes_clean = df_crashes_clean.dropna(subset=['CRASH_DATE'])
print(f"After converting and validating CRASH_DATE: {df_crashes_clean.shape}")


Original shape: (100000, 29)
After dropping missing COLLISION_ID: (100000, 29)
After dropping missing CRASH_DATE: (100000, 29)
After converting and validating CRASH_DATE: (100000, 29)


In [17]:
# Standardize BOROUGH column
df_crashes_clean['BOROUGH'] = df_crashes_clean['BOROUGH'].apply(standardize_borough)

# Fill missing boroughs with 'UNKNOWN' (we'll handle this later if needed)
borough_counts_after = df_crashes_clean['BOROUGH'].value_counts()
print("Borough distribution after standardization:")
print(borough_counts_after)


Borough distribution after standardization:
BOROUGH
UNKNOWN          30771
BROOKLYN         22165
QUEENS           18548
MANHATTAN        15460
BRONX            10162
STATEN ISLAND     2894
Name: count, dtype: int64


In [18]:
# Clean contributing factors
for col in ['CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2']:
    if col in df_crashes_clean.columns:
        df_crashes_clean[col] = df_crashes_clean[col].apply(clean_contributing_factor)

print("Contributing factors cleaned.")


Contributing factors cleaned.


### 3.2 Detect and Address Outliers


In [19]:
# Check for outliers in numeric columns
numeric_cols_to_check = ['LATITUDE', 'LONGITUDE', 'NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED']

for col in numeric_cols_to_check:
    if col in df_crashes_clean.columns:
        outliers = detect_outliers_iqr(df_crashes_clean[col].dropna())
        print(f"{col}: {outliers.sum()} outliers detected ({outliers.sum()/len(df_crashes_clean)*100:.2f}%)")


LATITUDE: 335 outliers detected (0.34%)
LONGITUDE: 2299 outliers detected (2.30%)


In [20]:
# Validate and clean coordinates
if 'LATITUDE' in df_crashes_clean.columns and 'LONGITUDE' in df_crashes_clean.columns:
    # Create validation mask
    valid_coords = df_crashes_clean.apply(
        lambda row: validate_coordinates(row['LATITUDE'], row['LONGITUDE']),
        axis=1
    )

    print(f"Valid coordinates: {valid_coords.sum()} ({valid_coords.sum()/len(df_crashes_clean)*100:.2f}%)")
    print(f"Invalid coordinates: {(~valid_coords).sum()} ({(~valid_coords).sum()/len(df_crashes_clean)*100:.2f}%)")

    # Option: Keep invalid coordinates but mark them, or drop them
    # For now, we'll keep them but they won't be used in map visualizations


Valid coordinates: 88717 (88.72%)
Invalid coordinates: 11283 (11.28%)


### 3.3 Standardize Formats


In [21]:
# Extract temporal features
df_crashes_clean['YEAR'] = df_crashes_clean['CRASH_DATE'].dt.year
df_crashes_clean['MONTH'] = df_crashes_clean['CRASH_DATE'].dt.month
df_crashes_clean['DAY_OF_WEEK'] = df_crashes_clean['CRASH_DATE'].dt.day_name()
df_crashes_clean['DAY_OF_WEEK_NUM'] = df_crashes_clean['CRASH_DATE'].dt.dayofweek

# Extract hour from CRASH_TIME
if 'CRASH_TIME' in df_crashes_clean.columns:
    df_crashes_clean['HOUR'] = pd.to_datetime(
        df_crashes_clean['CRASH_TIME'],
        format='%H:%M:%S',
        errors='coerce'
    ).dt.hour

print("Temporal features extracted.")
print(f"Year range: {df_crashes_clean['YEAR'].min()} - {df_crashes_clean['YEAR'].max()}")


Temporal features extracted.
Year range: 2012 - 2025


### 3.4 Remove Duplicates


In [22]:
# Check for duplicate COLLISION_IDs (should be unique per crash)
duplicate_ids = df_crashes_clean[df_crashes_clean.duplicated(subset=['COLLISION_ID'], keep=False)]
print(f"Duplicate COLLISION_IDs: {len(duplicate_ids)}")

if len(duplicate_ids) > 0:
    # Keep first occurrence
    df_crashes_clean = df_crashes_clean.drop_duplicates(subset=['COLLISION_ID'], keep='first')
    print(f"After removing duplicates: {df_crashes_clean.shape}")

# Check for completely duplicate rows
duplicate_rows = df_crashes_clean.duplicated().sum()
print(f"Completely duplicate rows: {duplicate_rows}")
if duplicate_rows > 0:
    df_crashes_clean = df_crashes_clean.drop_duplicates()
    print(f"After removing duplicate rows: {df_crashes_clean.shape}")


Duplicate COLLISION_IDs: 0
Completely duplicate rows: 0


## 4. Data Integration

Integrate crashes data with persons dataset using COLLISION_ID.


In [23]:
# Explore persons dataset
print("=== PERSONS DATASET OVERVIEW ===")
print(f"Shape: {df_persons.shape}")
print(f"Unique COLLISION_IDs: {df_persons['COLLISION_ID'].nunique()}")
print(f"\nMissing values:\n{df_persons.isnull().sum().sort_values(ascending=False).head(10)}")
print(f"\nPerson types:\n{df_persons['PERSON_TYPE'].value_counts() if 'PERSON_TYPE' in df_persons.columns else 'N/A'}")


=== PERSONS DATASET OVERVIEW ===
Shape: (5819475, 21)
Unique COLLISION_IDs: 1590325

Missing values:
CONTRIBUTING_FACTOR_2    5718669
CONTRIBUTING_FACTOR_1    5718536
PED_ACTION               5717258
PED_LOCATION             5717157
SAFETY_EQUIPMENT         3030154
EJECTION                 2827390
POSITION_IN_VEHICLE      2826933
EMOTIONAL_STATUS         2730419
BODILY_INJURY            2730376
COMPLAINT                2730369
dtype: int64

Person types:
PERSON_TYPE
Occupant           5586940
Pedestrian          142146
Bicyclist            78182
Other Motorized      12207
Name: count, dtype: int64


In [26]:
# ---- 4. Persons aggregation (fast + compatible with merge) ----

# Keep only persons for collisions that are in the cleaned crashes sample
valid_ids = df_crashes_clean['COLLISION_ID'].unique()
df_persons_small = df_persons[df_persons['COLLISION_ID'].isin(valid_ids)].copy()

# Make sure COLLISION_ID has same dtype in both tables
df_persons_small['COLLISION_ID'] = df_persons_small['COLLISION_ID'].astype(
    df_crashes_clean['COLLISION_ID'].dtype
)

# Cast once (not inside groupby) for the string columns
df_persons_small['PERSON_TYPE'] = df_persons_small['PERSON_TYPE'].astype('string')
df_persons_small['PERSON_INJURY'] = df_persons_small['PERSON_INJURY'].astype('string')

import pandas as pd

def join_unique(x):
    """Join up to 5 unique non-null values into a comma-separated string."""
    vals = pd.unique(x.dropna())
    return ', '.join(vals[:5])

# Aggregate persons data by COLLISION_ID
persons_agg = df_persons_small.groupby('COLLISION_ID', as_index=False).agg(
    PERSON_TYPES    = ('PERSON_TYPE',   join_unique),
    PERSON_INJURIES = ('PERSON_INJURY', join_unique),
    AVG_PERSON_AGE  = ('PERSON_AGE', 'mean'),
    MIN_PERSON_AGE  = ('PERSON_AGE', 'min'),
    MAX_PERSON_AGE  = ('PERSON_AGE', 'max'),
    PERSON_COUNT    = ('PERSON_AGE', 'size')   # number of persons in this collision
)

print(f"Aggregated persons data shape: {persons_agg.shape}")
persons_agg.head()


Aggregated persons data shape: (71752, 7)


,COLLISION_ID,PERSON_TYPES,PERSON_INJURIES,AVG_PERSON_AGE,MIN_PERSON_AGE,MAX_PERSON_AGE,PERSON_COUNT
0,228,Pedestrian,Injured,58.0,58.0,58.0,1
1,324,Pedestrian,Injured,32.0,32.0,32.0,1
2,638,Pedestrian,Injured,57.5,56.0,59.0,2
3,864,Bicyclist,Injured,29.0,29.0,29.0,1
4,868,Occupant,Injured,39.5,25.0,54.0,2


In [27]:
# Perform left join to integrate datasets
df_integrated = df_crashes_clean.merge(
    persons_agg,
    on='COLLISION_ID',
    how='left'
)

print(f"Integrated dataset shape: {df_integrated.shape}")
print(f"Rows with person data: {df_integrated['PERSON_TYPES'].notna().sum()}")
print(f"Rows without person data: {df_integrated['PERSON_TYPES'].isna().sum()}")

df_integrated.head()


Integrated dataset shape: (100000, 39)
Rows with person data: 71752
Rows without person data: 28248


,CRASH_DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,YEAR,MONTH,DAY_OF_WEEK,DAY_OF_WEEK_NUM,PERSON_TYPES,PERSON_INJURIES,AVG_PERSON_AGE,MIN_PERSON_AGE,MAX_PERSON_AGE,PERSON_COUNT
0,2024-11-08,15:45,BROOKLYN,11201,40.687363,-73.990036,"(40.687363, -73.990036)",SMITH ST,DEAN ST,NaN,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Passing or Lane Usage Improper,NaN,NaN,NaN,4770105,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,2024,11,Friday,4,"Occupant, Other Motorized","Unspecified, Injured",31.0,7.0,44.0,4.0
1,2019-11-10,14:00,QUEENS,11366,40.729270,-73.788290,"(40.72927, -73.78829)",NaN,NaN,75-44 181 STREET,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4238500,Sedan,Sedan,NaN,NaN,NaN,2019,11,Sunday,6,Occupant,Unspecified,55.5,38.0,73.0,4.0
2,2019-04-05,14:00,UNKNOWN,NaN,40.704304,-73.803150,"(40.704304, -73.80315)",90 AVENUE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Turning Improperly,Unspecified,NaN,NaN,NaN,4109437,Box Truck,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2019,4,Friday,4,Occupant,Unspecified,44.0,42.0,48.0,3.0
3,2022-03-09,11:14,MANHATTAN,10021,0.000000,0.000000,"(0.0, 0.0)",NaN,NaN,218 EAST 74 STREET,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4508714,Convertible,Sedan,NaN,NaN,NaN,2022,3,Wednesday,2,Occupant,Unspecified,46.5,38.0,55.0,4.0
4,2016-09-17,23:15,UNKNOWN,NaN,40.704388,-73.994576,"(40.704388, -73.994576)",BROOKLYN BRIDGE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3522798,Sedan,NaN,NaN,NaN,NaN,2016,9,Saturday,5,Occupant,Unspecified,10.0,0.0,20.0,2.0


## 5. Post-Integration Cleaning

After joining, we need to handle new missing values and inconsistencies.


In [28]:
# Check for new missing values introduced by the join
print("=== MISSING VALUES AFTER INTEGRATION ===")
missing_after = df_integrated.isnull().sum().sort_values(ascending=False)
print(missing_after[missing_after > 0].head(15))


=== MISSING VALUES AFTER INTEGRATION ===
VEHICLE TYPE CODE 5              99532
CONTRIBUTING FACTOR VEHICLE 5    99520
VEHICLE TYPE CODE 4              98384
CONTRIBUTING FACTOR VEHICLE 4    98322
VEHICLE TYPE CODE 3              93048
CONTRIBUTING FACTOR VEHICLE 3    92742
OFF STREET NAME                  82078
CROSS STREET NAME                38595
ZIP CODE                         30783
MIN_PERSON_AGE                   29441
MAX_PERSON_AGE                   29441
AVG_PERSON_AGE                   29441
PERSON_COUNT                     28248
PERSON_TYPES                     28248
PERSON_INJURIES                  28248
dtype: int64


In [29]:
# Handle missing values from join
# For person-related fields, missing means no person data was available
# We'll fill with appropriate defaults
df_integrated['PERSON_TYPES'] = df_integrated['PERSON_TYPES'].fillna('UNKNOWN')
df_integrated['PERSON_INJURIES'] = df_integrated['PERSON_INJURIES'].fillna('UNKNOWN')
df_integrated['PERSON_COUNT'] = df_integrated['PERSON_COUNT'].fillna(0)
df_integrated['AVG_PERSON_AGE'] = df_integrated['AVG_PERSON_AGE'].fillna(0)

print("Missing values handled for person-related fields.")


Missing values handled for person-related fields.


In [30]:
# Check for data type mismatches
print("=== DATA TYPES ===")
print(df_integrated.dtypes.value_counts())

# Ensure numeric columns are properly typed
numeric_cols_final = ['NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED',
                      'NUMBER_OF_PEDESTRIANS_INJURED', 'NUMBER_OF_PEDESTRIANS_KILLED',
                      'NUMBER_OF_CYCLIST_INJURED', 'NUMBER_OF_CYCLIST_KILLED',
                      'NUMBER_OF_MOTORIST_INJURED', 'NUMBER_OF_MOTORIST_KILLED']

for col in numeric_cols_final:
    if col in df_integrated.columns:
        df_integrated[col] = pd.to_numeric(df_integrated[col], errors='coerce').fillna(0)

print("Numeric columns standardized.")


=== DATA TYPES ===
object            18
float64            8
int64              7
int32              3
string[python]     2
datetime64[ns]     1
Name: count, dtype: int64
Numeric columns standardized.


## 6. Visualizations

### 6.1 Temporal Patterns


In [31]:
# Crashes by year
yearly_counts = df_integrated.groupby('YEAR').size().reset_index(name='count')
fig = px.line(yearly_counts, x='YEAR', y='count',
              title='Number of Crashes by Year',
              markers=True)
fig.update_layout(height=400)
fig.show()


In [32]:
# Crashes by hour of day
if 'HOUR' in df_integrated.columns:
    hourly_counts = df_integrated['HOUR'].value_counts().sort_index()
    fig = px.bar(x=hourly_counts.index, y=hourly_counts.values,
                 title='Crashes by Hour of Day',
                 labels={'x': 'Hour', 'y': 'Number of Crashes'})
    fig.update_layout(height=400)
    fig.show()


In [34]:
# Heatmap: Day of week vs Hour
if 'HOUR' in df_integrated.columns and 'DAY_OF_WEEK' in df_integrated.columns:
    heatmap_data = df_integrated.groupby(['DAY_OF_WEEK', 'HOUR']).size().reset_index(name='count')
    heatmap_pivot = heatmap_data.pivot(index='DAY_OF_WEEK', columns='HOUR', values='count')

    # Reorder days
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    heatmap_pivot = heatmap_pivot.reindex([d for d in day_order if d in heatmap_pivot.index])

    fig = px.imshow(heatmap_pivot,
                    labels=dict(x="Hour", y="Day of Week", color="Crashes"),
                    title="Crashes Heatmap: Day of Week vs Hour",
                    aspect="auto")
    fig.update_layout(height=500)
    fig.show()


### 6.2 Spatial Distribution


In [35]:
# Map visualization (sample for performance)
map_df = df_integrated[['LATITUDE', 'LONGITUDE', 'BOROUGH']].dropna()
if len(map_df) > 10000:
    map_df = map_df.sample(n=10000, random_state=42)

fig = px.scatter_mapbox(
    map_df,
    lat='LATITUDE',
    lon='LONGITUDE',
    color='BOROUGH',
    zoom=10,
    height=600,
    mapbox_style='open-street-map',
    title='Spatial Distribution of Crashes'
)
fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))
fig.show()


### 6.3 Borough Analysis


In [36]:
# Crashes by borough
borough_counts = df_integrated['BOROUGH'].value_counts()
fig = px.bar(x=borough_counts.index, y=borough_counts.values,
             title='Total Crashes by Borough',
             labels={'x': 'Borough', 'y': 'Number of Crashes'})
fig.update_layout(height=400)
fig.show()


In [37]:
# Calculate crash rate per capita
crash_rates = calculate_crash_rate_per_capita(df_integrated)
fig = px.bar(x=crash_rates.index, y=crash_rates.values,
             title='Crash Rate per 100,000 Population by Borough',
             labels={'x': 'Borough', 'y': 'Crashes per 100k'})
fig.update_layout(height=400)
fig.show()


### 6.4 Contributing Factors


In [38]:
# Top contributing factors
if 'CONTRIBUTING_FACTOR_VEHICLE_1' in df_integrated.columns:
    factor_counts = df_integrated['CONTRIBUTING_FACTOR_VEHICLE_1'].value_counts().head(15)
    fig = px.bar(x=factor_counts.values, y=factor_counts.index,
                 orientation='h',
                 title='Top 15 Contributing Factors',
                 labels={'x': 'Number of Crashes', 'y': 'Factor'})
    fig.update_layout(height=500)
    fig.show()


### 6.5 Severity Analysis


In [44]:
# Standardize column names for injuries/fatalities
df_integrated = df_integrated.rename(columns={
    'NUMBER OF PERSONS INJURED': 'NUMBER_OF_PERSONS_INJURED',
    'NUMBER OF PERSONS KILLED':  'NUMBER_OF_PERSONS_KILLED'
})

if 'YEAR' not in df_integrated.columns:
    df_integrated['YEAR'] = df_integrated['CRASH_DATE'].dt.year


# Injuries and fatalities by year
yearly_severity = df_integrated.groupby('YEAR').agg({
    'NUMBER_OF_PERSONS_INJURED': 'sum',
    'NUMBER_OF_PERSONS_KILLED': 'sum'
}).reset_index()

fig = make_subplots(specs=[[{"secondary_y": False}]])
fig.add_trace(
    go.Scatter(x=yearly_severity['YEAR'], y=yearly_severity['NUMBER_OF_PERSONS_INJURED'],
              name='Injuries', mode='lines+markers'),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=yearly_severity['YEAR'], y=yearly_severity['NUMBER_OF_PERSONS_KILLED'],
              name='Fatalities', mode='lines+markers'),
    secondary_y=False
)
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Count", secondary_y=False)
fig.update_layout(title='Injuries and Fatalities Trends Over Years', height=400)
fig.show()


## 7. Research Questions Analysis

### Research Question 1: Which borough has the highest crash rate per capita?


In [45]:
# Already calculated above - display results
print("Crash Rate per 100,000 Population:")
print(crash_rates.sort_values(ascending=False))
print(f"\nBorough with highest crash rate: {crash_rates.idxmax()}")
print(f"Rate: {crash_rates.max():.2f} crashes per 100k population")


Crash Rate per 100,000 Population:
UNKNOWN          3.077100e+09
MANHATTAN        9.124976e+02
BROOKLYN         8.101024e+02
QUEENS           7.710778e+02
BRONX            6.900467e+02
STATEN ISLAND    5.837655e+02
dtype: float64

Borough with highest crash rate: UNKNOWN
Rate: 3077100000.00 crashes per 100k population


### Research Question 2: What are the temporal patterns of crashes?


In [49]:
# --- Standardize date/time column names on the integrated dataset ---

df_integrated = df_integrated.rename(columns={
    'CRASH DATE': 'CRASH_DATE',
    'CRASH TIME': 'CRASH_TIME'
})

# --- Create temporal feature columns safely ---

import pandas as pd

# Ensure CRASH_DATE is datetime
df_integrated['CRASH_DATE'] = pd.to_datetime(df_integrated['CRASH_DATE'], errors='coerce')

# YEAR / MONTH / DAY_OF_WEEK
df_integrated['YEAR'] = df_integrated['CRASH_DATE'].dt.year
df_integrated['MONTH'] = df_integrated['CRASH_DATE'].dt.month
df_integrated['DAY_OF_WEEK'] = df_integrated['CRASH_DATE'].dt.day_name()

# Handle time if present
if 'CRASH_TIME' in df_integrated.columns:
    df_integrated['CRASH_TIME'] = pd.to_datetime(
        df_integrated['CRASH_TIME'], format='%H:%M', errors='coerce'
    )
    df_integrated['HOUR'] = df_integrated['CRASH_TIME'].dt.hour
else:
    # If for some reason time column is missing, still create HOUR as NaN
    df_integrated['HOUR'] = pd.NA

print(df_integrated[['CRASH_DATE','CRASH_TIME','HOUR','DAY_OF_WEEK','MONTH']].head())


  CRASH_DATE          CRASH_TIME  HOUR DAY_OF_WEEK  MONTH
0 2024-11-08 1900-01-01 15:45:00    15      Friday     11
1 2019-11-10 1900-01-01 14:00:00    14      Sunday     11
2 2019-04-05 1900-01-01 14:00:00    14      Friday      4
3 2022-03-09 1900-01-01 11:14:00    11   Wednesday      3
4 2016-09-17 1900-01-01 23:15:00    23    Saturday      9


In [50]:
# --- Create temporal feature columns ---

# Make sure crash date is datetime
df_integrated['CRASH_DATE'] = pd.to_datetime(df_integrated['CRASH_DATE'], errors='coerce')

# Extract month and day of week
df_integrated['MONTH'] = df_integrated['CRASH_DATE'].dt.month
df_integrated['DAY_OF_WEEK'] = df_integrated['CRASH_DATE'].dt.day_name()

# Extract hour from CRASH TIME
# First safely convert to datetime.time
df_integrated['CRASH_TIME'] = pd.to_datetime(df_integrated['CRASH_TIME'], format='%H:%M', errors='coerce')

df_integrated['HOUR'] = df_integrated['CRASH_TIME'].dt.hour

print(df_integrated[['CRASH_DATE','CRASH_TIME','HOUR','DAY_OF_WEEK','MONTH']].head())


  CRASH_DATE          CRASH_TIME  HOUR DAY_OF_WEEK  MONTH
0 2024-11-08 1900-01-01 15:45:00    15      Friday     11
1 2019-11-10 1900-01-01 14:00:00    14      Sunday     11
2 2019-04-05 1900-01-01 14:00:00    14      Friday      4
3 2022-03-09 1900-01-01 11:14:00    11   Wednesday      3
4 2016-09-17 1900-01-01 23:15:00    23    Saturday      9


### Research Question 3: Which contributing factors are most associated with fatalities?


In [46]:
# Analyze contributing factors for fatal crashes
fatal_crashes = df_integrated[df_integrated['NUMBER_OF_PERSONS_KILLED'] > 0]

if len(fatal_crashes) > 0 and 'CONTRIBUTING_FACTOR_VEHICLE_1' in fatal_crashes.columns:
    fatal_factors = fatal_crashes['CONTRIBUTING_FACTOR_VEHICLE_1'].value_counts().head(10)

    fig = px.bar(x=fatal_factors.values, y=fatal_factors.index,
                 orientation='h',
                 title='Top Contributing Factors in Fatal Crashes',
                 labels={'x': 'Number of Fatal Crashes', 'y': 'Factor'})
    fig.update_layout(height=400)
    fig.show()

    # Calculate fatality rate by factor
    factor_fatality_rate = {}
    for factor in fatal_factors.index:
        total_with_factor = len(df_integrated[df_integrated['CONTRIBUTING_FACTOR_VEHICLE_1'] == factor])
        fatal_with_factor = len(fatal_crashes[fatal_crashes['CONTRIBUTING_FACTOR_VEHICLE_1'] == factor])
        factor_fatality_rate[factor] = (fatal_with_factor / total_with_factor * 100) if total_with_factor > 0 else 0

    print("\nFatality Rate by Contributing Factor (%):")
    for factor, rate in sorted(factor_fatality_rate.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"{factor}: {rate:.2f}%")


## Summary and Conclusions

### Key Findings:
1. **Data Quality**: The dataset contains over 2 million records with significant missing values, requiring careful cleaning.
2. **Temporal Patterns**: Crashes show distinct patterns by hour, day, and season.
3. **Spatial Distribution**: Crashes are not evenly distributed across boroughs.
4. **Contributing Factors**: Certain factors are more strongly associated with severe outcomes.

### Cleaning Decisions Justified:
- **Dropped missing COLLISION_ID and CRASH_DATE**: These are critical fields without which records are unusable.
- **Standardized BOROUGH**: Multiple variations of borough names needed standardization.
- **Kept invalid coordinates**: Rather than dropping, we mark them for exclusion from map visualizations.
- **Left join for integration**: Preserves all crash records even if person data is missing.

### Next Steps:
- Deploy interactive dashboard
- Further analysis on specific research questions
- Model development for predictive insights


In [47]:
# Save cleaned and integrated dataset (optional)
# Uncomment the following lines to save the processed data

# from google.colab import files  # For Colab download
# df_integrated.to_csv('integrated_data.csv', index=False)
# files.download('integrated_data.csv')  # Download from Colab

# Or save to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# df_integrated.to_csv('/content/drive/MyDrive/integrated_data.csv', index=False)

print(f"Final dataset shape: {df_integrated.shape}")
print("\nDataset is ready for analysis!")



Final dataset shape: (100000, 39)

Dataset is ready for analysis!
